In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pytrends.request import TrendReq
from pytrends import dailydata
import requests
import collections

from sklearn import linear_model
import statsmodels.api as sm

from backtesting.test import GOOG

/opt/anaconda3/lib/python3.8/site-packages/backtesting/_plotting.py:45: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [47]:
key_word = "debt"
start_year = 2014
start_month = 1
end_year = 2018
end_month = 12
trend = pd.DataFrame(dailydata.get_daily_data(key_word, start_year, start_month, end_year, end_month, wait_time = 0)[key_word])
trend = trend[trend.index.dayofweek == 1]
trend["change"] = trend.pct_change()

# grab DJIA data
djia = yf.download("DJIA", start = str(start_year)+"-"+str(start_month)+"-01", end = str(end_year)+"-"+str(end_month)+"-01")

# join trend data with DJIA data
joined = trend.merge(djia, left_on = trend.index, right_on = djia.index)
joined = joined.rename(columns = {"key_0": "Date"})

# grab the adj close price difference for each day
#joined['Diff'] = joined['Adj Close'].diff()
joined = joined.set_index(joined['Date'])

debt:2014-01-01 2014-01-31
debt:2014-02-01 2014-02-28
debt:2014-03-01 2014-03-31
debt:2014-04-01 2014-04-30
debt:2014-05-01 2014-05-31
debt:2014-06-01 2014-06-30
debt:2014-07-01 2014-07-31
debt:2014-08-01 2014-08-31
debt:2014-09-01 2014-09-30
debt:2014-10-01 2014-10-31
debt:2014-11-01 2014-11-30
debt:2014-12-01 2014-12-31
debt:2015-01-01 2015-01-31
debt:2015-02-01 2015-02-28
debt:2015-03-01 2015-03-31
debt:2015-04-01 2015-04-30
debt:2015-05-01 2015-05-31
debt:2015-06-01 2015-06-30
debt:2015-07-01 2015-07-31
debt:2015-08-01 2015-08-31
debt:2015-09-01 2015-09-30
debt:2015-10-01 2015-10-31
debt:2015-11-01 2015-11-30
debt:2015-12-01 2015-12-31
debt:2016-01-01 2016-01-31
debt:2016-02-01 2016-02-29
debt:2016-03-01 2016-03-31
debt:2016-04-01 2016-04-30
debt:2016-05-01 2016-05-31
debt:2016-06-01 2016-06-30
debt:2016-07-01 2016-07-31
debt:2016-08-01 2016-08-31
debt:2016-09-01 2016-09-30
debt:2016-10-01 2016-10-31
debt:2016-11-01 2016-11-30
debt:2016-12-01 2016-12-31
debt:2017-01-01 2017-01-31
d

In [48]:
joined

,Date,debt,change,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,,,
2014-01-07,2014-01-07,25.20,NaN,16429.019531,16629.410156,16409.429688,16530.939453,16530.939453,3511750000
2014-01-14,2014-01-14,32.39,0.285317,16261.990234,16425.939453,16205.669922,16373.860352,16373.860352,3353270000
2014-01-21,2014-01-21,23.79,-0.265514,16459.269531,16591.369141,16283.339844,16414.439453,16414.439453,3782470000
2014-01-28,2014-01-28,41.40,0.740227,15840.839844,16012.139648,15812.320312,15928.559570,15928.559570,3437830000
2014-02-04,2014-02-04,34.96,-0.155556,15372.799805,15551.530273,15284.280273,15445.240234,15445.240234,4068410000
...,...,...,...,...,...,...,...,...,...
2018-10-30,2018-10-30,42.63,-0.033333,24482.039062,24998.009766,24245.269531,24874.640625,24874.640625,5106380000
2018-11-06,2018-11-06,42.75,0.002815,25452.830078,25725.130859,25341.990234,25635.009766,25635.009766,3510860000
2018-11-13,2018-11-13,43.20,0.010526,25321.210938,25655.390625,25063.109375,25286.490234,25286.490234,4091440000


In [49]:
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

In [50]:
def ourFunc(df):
    return pd.Series(df['change'])

In [51]:
from backtesting import Strategy
from backtesting.lib import crossover


class ourStrat(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    #n1 = 10
    #n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        #self.sma1 = self.I(SMA, self.data.Close, self.n1)
        #self.sma2 = self.I(SMA, self.data.Close, self.n2)
        self.change = self.I(ourFunc, self.data)
        self.own = False
        #self.buy_from_last_week = False
        #self.sell_from_last_week = True
    
    def next(self):
        """
        if self.buy_from_last_week:
            self.buy()
            self.own = True
            self.buy_from_last_week = False
        
        elif self.sell_from_last_week:
            self.sell()
            self.own = False
            self.sell_from_last_week = False
        """
        
        if self.change[-1] > 0 and not self.own: #crossover(self.sma1, self.sma2):
            #self.position.close()
            self.buy()
            self.own = True

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif self.change[-1] <= 0 and self.own: #crossover(self.sma2, self.sma1):
            #self.position.close()
            self.sell()
            self.own = False

In [52]:
from backtesting import Backtest

bt = Backtest(joined, ourStrat, cash=100_000_000, commission=.002)
stats = bt.run()
stats

Start                     2014-01-07 00:00:00
End                       2018-11-27 00:00:00
Duration                   1785 days 00:00:00
Exposure Time [%]                     98.4314
Equity Final [$]                  1.60348e+08
Equity Peak [$]                   1.73817e+08
Return [%]                            60.3482
Buy & Hold Return [%]                 49.7116
Return (Ann.) [%]                       59.46
Volatility (Ann.) [%]                 41.9996
Sharpe Ratio                          1.41573
Sortino Ratio                         3.14882
Calmar Ratio                          4.11518
Max. Drawdown [%]                    -14.4489
Avg. Drawdown [%]                    -2.80264
Max. Drawdown Duration      420 days 00:00:00
Avg. Drawdown Duration       62 days 00:00:00
# Trades                                    1
Win Rate [%]                              100
Best Trade [%]                        59.4245
Worst Trade [%]                       59.4245
Avg. Trade [%]                    